# [CPSC 322](https://github.com/GonzagaCPSC322) Data Science Algorithms
[Gonzaga University](https://www.gonzaga.edu/)

[Gina Sprint](http://cs.gonzaga.edu/faculty/sprint/)

# Nearest Neighbor Classification
What are our learning objectives for this lesson?
* Learn about the kNN classification algorithm

Content used in this lesson is based upon information in the following sources:
* Dr. Shawn Bowers' Data Mining notes

## Warm-up Task(s)
1. (first, together) Create a new folder called ClassificationFun and create a `main.py`
    * Let's take some quick notes on programming interfaces for ML algorithms
1. (later) Add `mysimplelinearregressor.py` from https://github.com/GonzagaCPSC322/U4-Supervised-Learning/tree/master/ClassificationFun
    * Read through the module
1. (later) In main.py. create a `MySimpleLinearRegressor` object and call `fit()` using our "y = 2x + some noise" data as the training data. Then try calling `predict()` for an unseen instance e.g. `[150]`
    Note: `X_train` and `X_test` need to be 2D... `X` is a feature "matrix")

## Today 
* Announcements
    * No MA this week 🎉
    * PA3 is due Tuesday. Questions?
    * Grad school/career prep advice talk tonight, 6pm Bollier 120
        * There will be pizza and IQ4 bonus point question content 🤓
    * New club: Zags AI Lab (ZAIL)
        * Hosting a coding challenge that opens Friday 10/4: https://zail-challenge-2024.pangeon.com/
* Unit testing of `MySimpleLinearRegressor`
* kNN overview and algorithm trace
* IQ3 last ~15 mins of class

## Warm-up Task(s) 
1. Accept, clone, and open PA4
1. Read through the `MySimpleLinearRegressionClassifier` docstrings in `myclassifiers.py`
    * What is the purpose of `discretizer`?
    * Functions in Python are objects, therefore a function name is a reference. See code snippet below
1. We are going to practice TDD (test-driven development... writing unit tests before writing the units themselves)

In [2]:
def double(x):
    return 2 * x

myfunc = double
print(type(double), type(myfunc), double == myfunc)
print(myfunc(5))

# often used with map, filter, reduce
nums = [3, 4, 5]
print("orig list:", nums)
# map a function to each elem in a list
print("map double to each elem:", list(map(myfunc, nums)))

# filter elems in a list by some criteria
def odd(x):
    return x % 2 == 1
print("filter each elem by odd:", list(filter(odd, nums)))

# reduce elems in a list to a single value
from functools import reduce
def total(total_so_far, x):
    return total_so_far + x
print("reduce elems to total value:", reduce(total, nums, 0))

<class 'function'> <class 'function'> True
10
orig list: [3, 4, 5]
map double to each elem: [6, 8, 10]
filter each elem by odd: [3, 5]
reduce elems to total value: 12


## Today
* Announcements
    * Nice job on IQ3, let's go over it.
    * IQ4 on Thursday on Jupyter Notebook/Markdown/Latex
    * PA3 is due tonight. Questions?
    * Work on PA4
    * RQ5 is posted and due **next Thursday 10/20** (so you can check your work after some examples of Naive Bayes we will do next week)
* PA4 starter code and hints

## PA4 Starter Code and Hints
1. kNN example trace (on iPad)
1. In ClassificationFun/main.py, write a unit test for a function called `compute_euclidean_distance()` that accepts two feature vectors and returns the Euclidean distance between them: $dist(v1, v2) = \sqrt{\sum_{i=1}^{n}(v1_i - v2_i)^{2}}$
    * Make up some values to call your function with
    * Test your function's output against SciPy's `euclidean()` function: https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.euclidean.html
    1. Write the `compute_euclidean_distance(v1, v2)` function and test it with `pytest main.py`
1. In ClassificationFun/main.py, consider the following labeled dataset, represented as the lists below. **Assume the two attributes have been normalized by scaling to a value between 0 and 10 (no additional normalization is necessary)**
    1. Copy and paste the following lists into your main.py
    1. Compute the distance between each train instance and the unseen instance: `[2, 3]`
        * Print out the distances and inspect them...
        * What are the k=3 closest neighbors of the following instance and how would you classify it?
            * How can you programmatically answer this question?
            * Hint: need a way to sort training instances by distance

```python
header = ["att1", "att2"]
X_train = [
    [3, 2],
    [6, 6],
    [4, 1],
    [4, 4],
    [1, 2],
    [2, 0],
    [0, 3],
    [1, 6]
]
y_train = ["no", "yes", "no", "no", "yes", "no", "yes", "yes"] # parallel to X_train
```

## Nearest Neighbor Classification
Nearest neighbor classification is typically used when attributes are continuous
* Can be modified for categorical data ...

Basic approach:
* Given an instance $i$ with $n - 1$ attributes (where the $n^{th}$ is class label)
* Find the "closest" instance $j$ to $i$ on the $n - 1$ attributes
* Use $j$'s class as the prediction for $i$

Example from book:
Given the data set

|a |b |c |d |e |f |Class|
|-|-|-|-|-|-|-|
|yes |no |no |6.4 |8.3 |low |negative|
|yes |yes |yes |18.2 |4.7 |high |positive|

What should this instance's classification be?

|yes |no |no |6.6 |8.0 |low |???|
|-|-|-|-|-|-|-|

Usually it isn't this easy!

## k Nearest Neighbors
Find the k nearest neighbors ...
* Usually find the k closest neighbors (instead of just closest)
* Then pick classification from among the top k

What are good values for the number of neighbors k?
* Often done experimentally (with a test set)
* Start with k = 1, determine "error rate" (more later)
* Repeat incrementing k
* Pick k with smallest (minimum) error rate
     * Often, larger the data (training) set, the larger the k

Lets say we found the k nearest neighbors for an instance ...

Q: What are ways we could pick the class?
* Most frequent occurring class
* Weighted "average" (based on the relative closest of the k)

Note: can use k-NN for regression if, e.g., return the mean of the label values

## Distance Functions
k-NN works by calculating distances between instances
* Many possible ways to do this ... generalized through "distance measures"
* For two points $x$ and $y$, the distance between them is given by $dist(x,y)$

Properties of distance measures (metrics)
1. $\forall x$, $dist(x,x) = 0$
    * The distance of any point x from itself is zero
2. $\forall xy$, $dist(x,y) = dist(y,x)$
    * Symmetry
3. $\forall xyz$, $dist(x,y) \leq dist(x,z) + dist(z,y)$
    * Triangle equality
    * "Shortest distance between any two points is a straight line"

Euclidean Distance is most often used: Given an instance, treat it as a "vector" in n space
* Use Pythagoras' Theorem to find distance between them

For example:
* Given two points (i.e., rows) with $n = 2$: $(x_1, y_1)$ and $(x_2, y_2)$
* The length (i.e., distance) of the straight line joining the points is:

$$\sqrt{(x_1 - x_2)^{2} + (y_1 - y_2)^{2}}$$

* Euclidean $n$-space
    * For rows A = $(a_1, a_2,..., a_n)$ and $B = (b_1, b_2,..., b_n)$ with $n$ attributes
$$\sqrt{(a_1 - b_1)^{2} + (a_2 - b_2)^{2} +...+ (a_n - b_n)^{2}}$$
        * Which is:
$$\sqrt{\sum_{i=1}^{n}(a_i - b_i)^{2}}$$

Other examples are described in the book (e.g., Manhattan "city block" distance)

Q: Do you see any possible issues with Euclidean distance?
* Larger values tend to dominate smaller ones
* Can degrade into a few attributes driving the distances
    * e.g., [Mileage=18,457, Doors=2, Age=12]
    
One solution is to scale all values between 0 and 1 ("min-max" normalization)
* Use the formula: `(x - min(xs)) / ((max(xs) - min(xs)) * 1.0)`

Q: How can we deal with categorical values?
* $dist(v_1, v_2) = 0$ if values $v_1 = v_2$
    * Same values
* For nominal values, $dist(v_1, v_2) = 1$ if $v_1 \neq v_2$
    * Different values
* For ordinal values, assign to 1 or use the "distance"

Q: What do we do about missing values?
* Don't have missing values
    * Clean the data first
* Be conservative
    * Assuming normalized
    * If only one value missing:
        * Assume the maximum possible distance
        * If nominal use the maximum distance (i.e., 1)
        * If ordinal use either 1 or furthest distance from known value
    * otherwise, if both values missing, use the maximum distance (e.g., 1)

Distance-based metrics imply equal weighting of attributes
* Sometimes can perform better with attribute weights (i.e., some attributes worth more)
* "Feature reduction" (not using certain attributes) can also help with redundant or "noisy" attributes

## Basic k-NN Algorithm
```
Input: X_train, y_train, X_test, k
row_distances = []
y_predicted = []
for test_instance in X_test:
    for train_instance in X_train:
        d = distance(train_instance, test_instance)
        row_distances.append([d, row])
    top_k_instances = get_top_k_instances(row_distances, k) # TODO: need to write this function
    prediction = select_class_label(top_k_instances)
    y_predicted.append(prediction)
```

## kNN Example 1
Example adapted from [this kNN example](https://people.revoledu.com/kardi/tutorial/KNN/KNN_Numerical-example.html)

Suppose we have the following dataset that has two attributes (acid durability and strength) and a class attribute (whether a special paper tissue is good or not):

|Acid durability (seconds)|Strength (kg/square meter)|Classification|
|-|-|-|
|7|7|Bad|
|7|4|Bad|
|3|4|Good|
|1|4| Good|

Now the factory produces a new paper tissue with acid durability = 3 seconds and strength = 7 kg/square meter. Can we predict what the classification of this new tissue is? 

Use kNN with $k$ = 3. Steps:
1. Normalize
1. Compute distance of each training instance to the test instance
1. Determine the majority classification of the $k$ closest instances... this is your prediction for the test instance

    
### Some Notes
Q: What happens if there are ties in the top-k distances (get_top_k)? E.g., which are top 3 in: [[.28,$r_1$],[.33,$r_2$],[.33,$r_3$],[.33,$r_4$],[.37,$r_5$]]?
* Different options ... e.g.:
    * Randomly select from ties
    * Do top-k distances (instead of instances)
    * Ignore ties (in case above, just use $r_1$ and $r_2$)

Nearest doesn't imply near
* top-k instances might not be that close to the instance being classified
* Especially true as the number of attributes ("dimensions") increases
    * An example of the "curse of dimensionality"
* Again, have to use common sense and an understanding of the dataset

## kNN Example 2
Consider the following labeled dataset, where result denotes class information and the remaining columns have continuous values. Assume the data has been normalized by scaling to a value between 0 and 10.

|att1|att2|result|
|-|-|-|
|3|2|no|
|6|6|yes|
|4|1|no|
|4|4|no|
|1|2|yes|
|2|0|no|
|0|3|yes|
|1|6|yes|

Assume we have the following instance to classify using $k$-NN for $k$ = 3 with "majority voting" to determine the class label from the k closest neighbors. 

|att1|att2|result|
|-|-|-|
|2|3|?|

1. What are the three closest neighbors of the following instance?
1. How would you classify it?

### Efficiency issues
Q: Is k-NN efficient? Can you find any efficiency issues?
* Given a training set with $D$ instances and $k = 1$
* $O(D)$ comparisons needed to classify a given instance

Q: Can you think of any ways to improve the efficiency?
1. Use search trees
    * Presort and arrange instances into a search tree
    * Can reduce comparisons to $O(log D)$
2. Check each training instance in parallel
    * Gives $O(1)$ comparisons
3. Editing/Pruning
    * Filter or remove training tuples that prove useless
    * Reduces size of $D$